# Exploring Dandiset 000563: Allen Institute Openscope - Barcoding

This notebook explores Dandiset 000563, which contains extracellular electrophysiology data related to "barcoding" neural responses to visual stimuli in the mouse brain.

**Note:** This notebook was AI-generated and has not been fully verified. Please exercise caution when interpreting the code or results.

## Dandiset Overview

Dandiset 000563, titled "Allen Institute Openscope - Barcoding", focuses on neural responses to white noise flicker visual stimuli, which can produce "barcode"-like patterns in spike rasters. This dataset explores whether these patterns could be used as identifiers of discrete cell types.

The Dandiset can be accessed at: https://dandiarchive.org/dandiset/000563/0.250311.2145

The data includes Neuropixels recordings throughout the mouse brain.

This notebook will demonstrate how to:
- Load basic information about the Dandiset using the DANDI API.
- Access one of the NWB files in the Dandiset using a remote URL.
- Explore the metadata and structure of the NWB file.
- Load and visualize some of the electrophysiology data (LFP).

## Required Packages

This notebook requires the following Python packages:
- `dandi`
- `pynwb`
- `h5py`
- `remfile`
- `numpy`
- `matplotlib`
- `seaborn`
- `pandas`
- `itertools`

In [1]:
# Load necessary libraries
from itertools import islice
from dandi.dandiapi import DandiAPIClient
import pynwb
import h5py
import remfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set seaborn theme for visualizations (excluding images)
sns.set_theme()

## Loading the Dandiset

We can use the DandiAPIClient to connect to the DANDI archive and retrieve information about the Dandiset.

In [2]:
# Connect to DANDI archive
client = DandiAPIClient()
dandiset = client.get_dandiset("000563", "0.250311.2145")

# Print basic information about the Dandiset
metadata = dandiset.get_raw_metadata()
print(f"Dandiset name: {metadata['name']}")
print(f"Dandiset URL: {metadata['url']}")

# List some assets in the Dandiset
assets = dandiset.get_assets()
print("\nFirst 5 assets:")
for i, asset in enumerate(islice(assets, 5)):
    print(f"- {asset.path} (ID: {asset.identifier})")
    if i == 0:
        # Store the first asset path and ID for later use
        first_asset_path = asset.path
        first_asset_id = asset.identifier

Dandiset name: Allen Institute Openscope - Barcoding
Dandiset URL: https://dandiarchive.org/dandiset/000563/0.250311.2145

First 5 assets:
- sub-699241/sub-699241_ses-1318772854_probe-1_ecephys.nwb (ID: 55148cf3-9d15-4d05-a190-4a73e007e48d)
- sub-699241/sub-699241_ses-1318772854_probe-3_ecephys.nwb (ID: 1b6325a9-1fc2-4fed-999c-d7c3aeb7a7a1)
- sub-699241/sub-699241_ses-1318772854_probe-4_ecephys.nwb (ID: 6e4d0047-ca6b-4d29-b150-bc9e2c445646)
- sub-699241/sub-699241_ses-1318772854_probe-0_ecephys.nwb (ID: 54cef618-72de-4c69-b441-8bd5e26049c5)
- sub-699241/sub-699241_ses-1318772854_ogen.nwb (ID: c3bbf094-904e-43b7-83d5-be5a8bf3826f)


## Loading an NWB file

This Dandiset contains multiple NWB files, primarily containing electrophysiology (`_ecephys.nwb`) and optogenetics (`_ogen.nwb`) data from different subjects and sessions.

We will load the following NWB file for demonstration:

`{first_asset_path}`

We can access this file directly using its remote URL derived from its Asset ID.

In [3]:
# Construct the URL for the selected asset
nwb_url = f"https://api.dandiarchive.org/api/assets/{first_asset_id}/download/"
print(f"Loading NWB file from URL: {nwb_url}")

# Load the NWB file remotely
remote_file = remfile.File(nwb_url)
h5_file = h5py.File(remote_file)
io = pynwb.NWBHDF5IO(file=h5_file)
nwb = io.read()

# We won't display the full nwb object as it can be very large.
print("\nNWB file loaded successfully.")
print(f"Session description: {nwb.session_description}")
print(f"Session ID: {nwb.session_id}")

Loading NWB file from URL: https://api.dandiarchive.org/api/assets/55148cf3-9d15-4d05-a190-4a73e007e48d/download/


/home/magland/miniconda3/envs/dev/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/magland/miniconda3/envs/dev/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/magland/miniconda3/envs/dev/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."



NWB file loaded successfully.
Session description: LFP data and associated info for one probe
Session ID: 1318772854


## Exploring NWB File Contents and Metadata

Let's look at some of the key sections and metadata available in this NWB file.

### Acquisition

The `acquisition` section contains the raw data recorded during the experiment. In this file, we expect to find the LFP data.

In [4]:
# Explore the acquisition section
print("Acquisition keys:")
for key in nwb.acquisition.keys():
    print(f"- {key}")

# Access the LFP data ElectricalSeries
if "probe_0_lfp" in nwb.acquisition and "probe_0_lfp_data" in nwb.acquisition["probe_0_lfp"].electrical_series:
    lfp_electrical_series = nwb.acquisition["probe_0_lfp"].electrical_series["probe_0_lfp_data"]
    print(f"\nLFP data shape: {lfp_electrical_series.data.shape}")
    print(f"LFP data units: {lfp_electrical_series.unit}")
    print(f"LFP timestamps shape: {lfp_electrical_series.timestamps.shape}")
else:
    print("\nCould not find expected LFP data in acquisition.")
    lfp_electrical_series = None # Ensure lfp_electrical_series is None if not found

Acquisition keys:
- probe_1_lfp
- probe_1_lfp_data

Could not find expected LFP data in acquisition.


### Electrodes

The `electrodes` table provides metadata about each recording channel, such as their location and grouping.

In [5]:
# Explore the electrodes table
if nwb.electrodes:
    print("\nElectrodes table columns:")
    for col in nwb.electrodes.colnames:
        print(f"- {col}")

    # Display the first few rows of the electrodes table
    electrodes_df = nwb.electrodes.to_dataframe()
    print("\nFirst 5 rows of the electrodes table:")
    print(electrodes_df.head())


Electrodes table columns:
- location
- group
- group_name
- probe_vertical_position
- probe_horizontal_position
- probe_id
- local_index
- valid_data
- x
- y
- z
- imp
- filtering

First 5 rows of the electrodes table:
     location                                              group group_name  \
id                                                                            
1002      Eth  probeC abc.EcephysElectrodeGroup at 0x13027543...     probeC   
1006      Eth  probeC abc.EcephysElectrodeGroup at 0x13027543...     probeC   
1010       LP  probeC abc.EcephysElectrodeGroup at 0x13027543...     probeC   
1014       LP  probeC abc.EcephysElectrodeGroup at 0x13027543...     probeC   
1018       LP  probeC abc.EcephysElectrodeGroup at 0x13027543...     probeC   

      probe_vertical_position  probe_horizontal_position  probe_id  \
id                                                                   
1002                       40                         59         1   
1006            

### Subject

Information about the experimental subject is stored in the `subject` section.

In [6]:
# Explore subject metadata
if nwb.subject:
    print("\nSubject metadata:")
    print(f"- Subject ID: {nwb.subject.subject_id}")
    print(f"- Species: {nwb.subject.species}")
    print(f"- Genotype: {nwb.subject.genotype}")
    print(f"- Sex: {nwb.subject.sex}")
    print(f"- Age: {nwb.subject.age}")
    print(f"- Strain: {nwb.subject.strain}")


Subject metadata:
- Subject ID: 699241
- Species: Mus musculus
- Genotype: Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt
- Sex: M
- Age: P124D
- Strain: Transgenic


## Exploring this NWB file on Neurosift

For a different way to explore the contents and structure of this NWB file directly in your web browser, you can use Neurosift:

https://neurosift.app/nwb?url={nwb_url}&dandisetId=000563&dandisetVersion=0.250311.2145

## Visualizing LFP Data

We can load a subset of the LFP data and its corresponding timestamps to visualize the activity over time. Since the data is large, we will load only a short segment for a few channels.

In [7]:
# Check if LFP data was successfully loaded
if lfp_electrical_series is not None:
    # Define the start and end index for the data subset
    start_index = 10000
    end_index = start_index + 5000 # Load 5000 samples
    
    # Define the channel indices to visualize
    channel_indices = [0, 10, 20, 30] # Visualize 4 channels

    # Load the data subset
    # We select rows by index and columns by index
    lfp_data_subset = lfp_electrical_series.data[start_index:end_index, channel_indices]

    # Load the corresponding timestamps subset
    lfp_timestamps_subset = lfp_electrical_series.timestamps[start_index:end_index]

    # Plot the LFP data subset
    plt.figure(figsize=(12, 6))
    for i, channel_index in enumerate(channel_indices):
        # Offset the channels vertically for better visibility
        offset = i * 500 # Adjust offset as needed
        plt.plot(lfp_timestamps_subset, lfp_data_subset[:, i] + offset, label=f'Channel Index {channel_index}')

    plt.xlabel("Time (s)")
    plt.ylabel(f"LFP Signal ({lfp_electrical_series.unit}) + Offset")
    plt.title("Subset of LFP Data from Selected Channels")
    plt.legend()
    plt.show()
else:
    print("LSP data not available for visualization.")

LSP data not available for visualization.


## Summary and Future Directions

This notebook demonstrated how to access and perform basic exploration and visualization of electrophysiology data from Dandiset 000563. We loaded Dandiset metadata, explored assets, accessed a specific NWB file remotely, examined its structure and metadata, and visualized a subset of the LFP data.

Possible future directions for analysis include:
- Exploring other NWB files in the Dandiset (e.g., optogenetics data).
- Downloading larger subsets of data for more extensive analysis.
- Applying signal processing techniques (e.g., filtering, spectral analysis) to the LFP data.
- Investigating the relationship between neural activity and the visual stimuli.
- Comparing data across different subjects or sessions.
- Analyzing spike times (if available in other NWB files in the dandiset) and their "barcode" patterns.